In [1]:
import pandas as pd
import numpy as np
import preprocess_helper
import os
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import enchant
checker = enchant.Dict("en_US")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\11099\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Use the processed ASRS output from `load_asrs()` in `preprocess_helper.py`.

In [2]:
# Dictionaries of acronym
# CASA = pd.read_csv('dictionaries/CASA.csv')
FAA = pd.read_csv('dictionaries/FAA.csv')
NASA = pd.read_csv('dictionaries/nasa_abbr.csv')
top_thirty = pd.read_csv('dictionaries/FAA Core Airports 2014.csv')

# Limit to top thirty airports
ASRS = preprocess_helper.load_asrs(load_saved=True)
top_thirty_bool = ASRS['tracon_code'].isin(top_thirty['tracon_key'])
ASRS_30 = ASRS[top_thirty_bool]

# Accident and incident counts of FAA and NTSB
acc_inc = pd.read_csv(r"E:\aviation_integrated\results\combined_vol_incident.csv")

D:\Anaconda\envs\aviation_integrated\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8,9,11,12,14,15,19,20,28,29,30,31,32,33,34,38,39,40,41,42,46,50,53,54,56,57,58,59,60,64,65,70,71,72,74,75,80,81,82,86,88,93,94,95,104) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


`narrative` is the combination of `narrative_report1` and `narrative_report2`.

In [3]:
# Utility function used for counting the acronyms in ASRS data

# Check if a word is spelled correctly 
def spell_check(word):
    return checker.check(word)

# Count the total occurrences of all dictionary acronyms in the text 
def count_total(word_count, dictionary):
    total = 0
    
    for word in dictionary:
        #if (word_count[word] != 0): print(word)
        total += word_count[word]
        
    return total

# Count the total unique occurrences of all dictionary acronyms in the text 
def count_unique(word_count, dictionary):
    total = 0
    
    for word in dictionary:
        if (word_count[word] != 0):
            total += 1
        
    return total

# Remove target symbols at both the start and the end of each string 
def strip_symbol(list_words, symbols):
    return list(map(lambda x: x.strip(symbols), list_words))

Note: It seems Jimmy had already removed all symbols in the `clean_string_and_tokenize` function.

**Update in 01/04/2022: Discard CASA dictionary temporarily and stop testing spelling check.** 

In [4]:
# Clean the field of acronyms

# CASA['acronym'] = CASA['acronym'].str.strip().str.lower()
# CASA['acronym'] = CASA['acronym'].str.replace(r'\(.+\)', '', regex=True)
# CASA.drop_duplicates(["acronym"], inplace=True, keep='first')
# CASA_cleaned = CASA[['acronym']]

FAA['acronym'] = FAA['acronym'].str.strip().str.lower()
FAA.drop_duplicates(["acronym"], inplace=True, keep='first')
FAA_cleaned = FAA[['acronym']]

NASA['acronym'] = NASA['acronym'].str.strip().str.lower()
NASA.drop_duplicates(["acronym"], inplace=True, keep='first')
NASA_cleaned = NASA[['acronym']]

top_thirty['tracon_key'] = top_thirty['tracon_key'].str.strip().str.upper()
top_thirty.drop_duplicates(["tracon_key"], inplace=True, keep='first')
top_thirty_cleaned = top_thirty[['tracon_key']]

# Remove stop words in the dictionaries
stop_words = set(stopwords.words('english'))

# bool_index = CASA_cleaned.isin(stop_words)['acronym'].values
# CASA_cleaned = CASA_cleaned[~bool_index]

bool_index = FAA_cleaned.isin(stop_words)['acronym'].values
FAA_cleaned = FAA_cleaned[~bool_index]

bool_index = NASA_cleaned.isin(stop_words)['acronym'].values
NASA_cleaned = NASA_cleaned[~bool_index]

# Filter out the words passing the spelling check (OBSOLETE)
# bool_index = np.array(list(map(spell_check, list(CASA_cleaned['acronym'].values))))
# CASA_cleaned = CASA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(FAA_cleaned['acronym'].values))))
# FAA_cleaned = FAA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(NASA_cleaned['acronym'].values))))
# NASA_cleaned = NASA_cleaned[~bool_index]

**Update in 01/07/2022: Use the top 30 airports listed in `FAA Core Airports 2014.csv`**

In [ ]:
# Count the occurrence of every distinct word in the text
words = ASRS_30['narrative'].str.lower().str.split()
words = words.apply(strip_symbol, symbols='{[(,:.')
words_count = words.apply(Counter)
acronyms = ASRS_30.loc[:, ['year', 'month', 'tracon_code', 'narrative', 'AtcAdvisoryMultCount']]
acronyms['word_counter'] = words_count

# Count the number of words
acronyms['word_count'] = words.apply(len)
acronyms['unique_word_count'] = words_count.apply(len)

# Sum the occurences of acronyms in different dictionaries respectively
# acronyms['CASA_total'] = acronyms['word_counter'].apply(count_occurrence, dictionary=CASA_cleaned['acronym'].values)
# acronyms['CASA_unique'] = acronyms['word_counter'].apply(count_unique, dictionary=CASA_cleaned['acronym'].values)

acronyms['FAA_total'] = acronyms['word_counter'].apply(count_total, dictionary=FAA_cleaned['acronym'].values)
acronyms['FAA_unique'] = acronyms['word_counter'].apply(count_unique, dictionary=FAA_cleaned['acronym'].values)

acronyms['NASA_total'] = acronyms['word_counter'].apply(count_total, dictionary=NASA_cleaned['acronym'].values)
acronyms['NASA_unique'] = acronyms['word_counter'].apply(count_unique, dictionary=NASA_cleaned['acronym'].values)

In [ ]:
acronyms.head()
acronyms.to_csv("quality_check/full_count.csv")

In [ ]:
acronyms.sort_values(['year', 'month', 'tracon_code'], inplace=True)
acronyms.head()

In [ ]:
# collapse by year-month-tracon and compute the sum and average per group
num_obs = pd.DataFrame(acronyms.groupby(['year', 'month', 'tracon_code']).size(), columns = ["num_obs"])
num_obs.reset_index(inplace = True)
sum_count = acronyms.groupby(['year', 'month', 'tracon_code']).sum()
sum_count.reset_index(inplace = True)
avg_count = acronyms.groupby(['year', 'month', 'tracon_code']).mean()
avg_count.reset_index(inplace = True)

agg_count = num_obs.merge(sum_count, on=['year', 'month', 'tracon_code'])
agg_count.rename(columns = {"word_count": "word_count_sum", "unique_word_count": "unique_word_count_sum",
                            "FAA_total": "FAA_total_sum", "FAA_unique": "FAA_unique_sum",
                            "NASA_total": "NASA_total_sum", "NASA_unique": "NASA_unique_sum",
                            "AtcAdvisoryMultCount": "AtcAdvisoryMultCount_sum"}, inplace=True)
agg_count = agg_count.merge(avg_count, on=['year', 'month', 'tracon_code'])
agg_count.rename(columns = {"word_count": "word_count_avg", "unique_word_count": "unique_word_count_avg",
                            "FAA_total": "FAA_total_avg", "FAA_unique": "FAA_unique_avg",
                            "NASA_total": "NASA_total_avg", "NASA_unique": "NASA_unique_avg",
                           "AtcAdvisoryMultCount": "AtcAdvisoryMultCount_avg", 'tracon_code': 'tracon'}, inplace=True)
agg_count

****
## Create Master Key

In [5]:
date = pd.date_range(start = '1/1/1988', end = '12/31/2019', freq = 'M')
date_df = pd.DataFrame(data = {'year': date.year, 'month': date.month})
master_key = top_thirty.merge(date_df, how = 'cross')
master_key = master_key.drop('ATADS_Type', axis = 1)
master_key = master_key.rename({'tracon_key': 'tracon'}, axis = 1)
master_key

,tracon,year,month
0,ATL,1988,1
1,ATL,1988,2
2,ATL,1988,3
3,ATL,1988,4
4,ATL,1988,5
...,...,...,...
11515,TPA,2019,8
11516,TPA,2019,9
11517,TPA,2019,10
11518,TPA,2019,11


In [6]:
master_key.to_csv("quality_check/master_key.csv")

## Merge FAA/NTSB with master key

In [7]:
# Limit NTSB/FAA to top 30
acc_inc_bool = acc_inc['airport_code'].isin(top_thirty['tracon_key'])
acc_inc_30 = acc_inc.loc[acc_inc_bool]
acc_inc_30 = acc_inc_30.drop("Unnamed: 0", axis = 1)
acc_inc_30 = acc_inc_30.rename({'airport_code': 'tracon'}, axis = 1)
acc_inc_30.to_csv('quality_check/acc_inc_30.csv')

In [8]:
faa_ntsb_merge = master_key.merge(acc_inc_30, left_on=['tracon', 'year', 'month'], right_on=['tracon', 'year', 'month'], how='left')
faa_ntsb_merge.to_csv('quality_check/merge_acc_inc.csv')
faa_ntsb_merge

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups
0,ATL,1988,1,0.0,0.0,1.0,0.0,1.0,1.0
1,ATL,1988,2,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,1988,3,1.0,0.0,4.0,1.0,5.0,4.0
3,ATL,1988,4,1.0,0.0,0.0,0.0,1.0,1.0
4,ATL,1988,5,0.0,2.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
11515,TPA,2019,8,0.0,1.0,0.0,0.0,1.0,1.0
11516,TPA,2019,9,0.0,0.0,2.0,0.0,2.0,2.0
11517,TPA,2019,10,NaN,NaN,NaN,NaN,NaN,NaN
11518,TPA,2019,11,NaN,NaN,NaN,NaN,NaN,NaN


Note: Perhaps mark the missing data with an indicator column?

In [9]:
faa_ntsb_merge = faa_ntsb_merge.fillna(0)

## Moving average of ASRS

In [15]:
# merge ASRS with master key
asrs_merge = master_key.merge(ASRS_30, left_on=['tracon', 'year', 'month'], right_on=['tracon_code', 'year', 'month'], how='left')
asrs_merge = asrs_merge.loc[:, ['year', 'month', 'tracon_code', 'narrative', 'AtcAdvisoryMultCount']]
asrs_merge['narrative'] = asrs_merge['narrative'].fillna('')
asrs_merge = asrs_merge.fillna(0)
asrs_merge.head(10)

,year,month,tracon_code,narrative,AtcAdvisoryMultCount
0,1988,1,ATL,apching the atl area a solid line of thunderst...,1.0
1,1988,1,ATL,my coplt and i mlg abc had just been clrd for ...,1.0
2,1988,1,ATL,while dsnding into the atl area on the 024 deg...,1.0
3,1988,2,ATL,established on ils rwy 27l atl on a 12 mile fi...,1.0
4,1988,2,ATL,atl apch ctlr gave a late turn on vector which...,1.0
5,1988,2,ATL,departed atl at xx30 lcl on 2 fri 88 and lande...,1.0
6,1988,2,ATL,acft departed atc clred on new dep proc to 400...,1.0
7,1988,2,ATL,we were on clbout from rwy 27r in atl clred to...,1.0
8,1988,3,ATL,f o was flying the acft and assigned a 060 deg...,1.0
9,1988,3,ATL,on taxi in at alt sbnd on ramp 3 right side no...,1.0


In [11]:
# create moving average window 
window_len = 6
lag = 2
windows = asrs_merge.rolling(window = window_len, min_periods=window_len)

In [12]:
# lists used to store the values from the moving window
num_word_sum = []
num_word_avg = []
num_unique_word_sum = []
num_unique_word_avg = []
FAA_total_sum = []
FAA_total_avg = []
FAA_unique_sum = []
FAA_unique_avg = []
NASA_total_sum = []
NASA_total_avg = []
NASA_unique_sum = []
NASA_unique_avg = []
AtcAdvisoryMultCount_sum = []
AtcAdvisoryMultCount_avg = []


for window in windows:
    size = window.shape[0]
    
    # create a dictionary of counts of words
    all_words = window['narrative'].str.cat(sep=' ')
    all_words = all_words.lower().split()
    all_words = [x.strip('{[(,:.') for x in all_words]
    words_count = Counter(all_words)
    
    # number of words
    num_word = len(all_words)
    num_word_sum.append(num_word)
    num_word_avg.append(num_word / size)
    
    # number of unique words
    num_unique_word = len(words_count)
    num_unique_word_sum.append(num_unique_word)
    num_unique_word_avg.append(num_unique_word / size)
    
    # number of FAA acronyms
    FAA_total = count_total(words_count, dictionary=FAA_cleaned['acronym'].values)
    FAA_total_sum.append(FAA_total)
    FAA_total_avg.append(FAA_total / size)
    
    # number of unique FAA acronyms
    FAA_unique = count_unique(words_count, dictionary=FAA_cleaned['acronym'].values)
    FAA_unique_sum.append(FAA_unique)
    FAA_unique_avg.append(FAA_unique / size)
    
    # number of NASA acronyms
    NASA_total = count_total(words_count, dictionary=NASA_cleaned['acronym'].values)
    NASA_total_sum.append(NASA_total)
    NASA_total_avg.append(NASA_total / size)
    
    # number of unique NASA acronyms
    NASA_unique = count_unique(words_count, dictionary=NASA_cleaned['acronym'].values)
    NASA_unique_sum.append(NASA_unique)
    NASA_unique_avg.append(NASA_unique / size)
    
    # number of atcadvisory splits
    num_splits = window['AtcAdvisoryMultCount'].sum()
    AtcAdvisoryMultCount_sum.append(num_splits)
    AtcAdvisoryMultCount_avg.append(num_splits / size)

num_word_sum = np.array(num_word_sum)
num_word_avg =  np.array(num_word_avg)
num_unique_word_sum = np.array(num_unique_word_sum)
num_unique_word_avg = np.array(num_unique_word_avg)
FAA_total_sum = np.array(FAA_total_sum)
FAA_total_avg = np.array(FAA_total_avg)
FAA_unique_sum = np.array(FAA_unique_sum)
FAA_unique_avg = np.array(FAA_unique_avg)
NASA_total_sum = np.array(NASA_total_sum)
NASA_total_avg = np.array(NASA_total_avg)
NASA_unique_sum = np.array(NASA_unique_sum)
NASA_unique_avg = np.array(NASA_unique_avg)
AtcAdvisoryMultCount_sum = np.array(AtcAdvisoryMultCount_sum)
AtcAdvisoryMultCount_avg = np.array(AtcAdvisoryMultCount_avg)

In [16]:
asrs_moving = asrs_merge.drop(['narrative', 'AtcAdvisoryMultCount'], axis = 1)
asrs_moving['num_word_sum'] = num_word_sum
asrs_moving['num_word_avg'] = num_word_avg
asrs_moving['num_unique_word_sum'] = num_unique_word_sum 
asrs_moving['num_unique_word_avg'] = num_unique_word_avg 
asrs_moving['FAA_total_sum'] = FAA_total_sum
asrs_moving['FAA_total_avg'] = FAA_total_avg
asrs_moving['FAA_unique_sum'] = FAA_unique_sum
asrs_moving['FAA_unique_avg'] = FAA_unique_avg
asrs_moving['NASA_total_sum'] = NASA_total_sum
asrs_moving['NASA_total_avg'] = NASA_total_avg
asrs_moving['NASA_unique_sum'] = NASA_unique_sum
asrs_moving['NASA_unique_avg'] = NASA_unique_avg
asrs_moving['AtcAdvisoryMultCount_sum'] = AtcAdvisoryMultCount_sum
asrs_moving['AtcAdvisoryMultCount_avg'] = AtcAdvisoryMultCount_avg
asrs_moving

,year,month,tracon_code,num_word_sum,num_word_avg,num_unique_word_sum,num_unique_word_avg,FAA_total_sum,FAA_total_avg,FAA_unique_sum,FAA_unique_avg,NASA_total_sum,NASA_total_avg,NASA_unique_sum,NASA_unique_avg,AtcAdvisoryMultCount_sum,AtcAdvisoryMultCount_avg
0,1988,1,ATL,200,200.000000,107,107.000000,7,7.000000,5,5.000000,26,26.000000,18,18.000000,1.0,1.000000
1,1988,1,ATL,874,437.000000,292,146.000000,27,13.500000,6,3.000000,109,54.500000,35,17.500000,2.0,1.000000
2,1988,1,ATL,1047,349.000000,343,114.333333,32,10.666667,7,2.333333,134,44.666667,46,15.333333,3.0,1.000000
3,1988,2,ATL,1223,305.750000,408,102.000000,49,12.250000,14,3.500000,156,39.000000,56,14.000000,4.0,1.000000
4,1988,2,ATL,1757,351.400000,516,103.200000,74,14.800000,19,3.800000,224,44.800000,69,13.800000,5.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39635,2019,8,0,98,16.333333,68,11.333333,1,0.166667,1,0.166667,0,0.000000,0,0.000000,1.0,0.166667
39636,2019,9,0,98,16.333333,68,11.333333,1,0.166667,1,0.166667,0,0.000000,0,0.000000,1.0,0.166667
39637,2019,10,0,98,16.333333,68,11.333333,1,0.166667,1,0.166667,0,0.000000,0,0.000000,1.0,0.166667
39638,2019,11,0,98,16.333333,68,11.333333,1,0.166667,1,0.166667,0,0.000000,0,0.000000,1.0,0.166667


In [17]:
asrs_merge.to_csv("quality_check/asrs_merge.csv")
asrs_moving.to_csv("quality_check/asrs_moving.csv")

****
## Create moving average of ASRS

In [ ]:
asrs_splits = [asrs_merge[asrs_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
faa_ntsb_splits = [faa_ntsb_merge[faa_ntsb_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized

### Toy example: ATL

In [ ]:
ATL = asrs_splits[0]

In [ ]:
window_len = 6
lag = 2
moving_avg = ATL.rolling(window_len, min_periods=window_len).mean()

In [ ]:
ATL.head(10)

1. Select columns to average and sum. No {year, month}
2. First get the window of the text, then do the manuplication and computation

In [ ]:
moving_avg.head(10)

In [ ]:
ATL.tail(10)

In [ ]:
moving_avg.tail(10)

In [ ]:
ATL.columns

In [ ]:
moving_avg = ATL.rolling(window_len, min_periods=window_len).sum()
empty_row_df = pd.DataFrame(data = [pd.Series(index=ATL.columns) for _ in range(lag)])
empty_row_df = empty_row_df.drop('tracon', axis = 1)
moving_avg = pd.concat([empty_row_df, moving_avg])
moving_avg = moving_avg.drop(['year', 'month'], axis = 1)
moving_avg = moving_avg.reset_index().drop('index', axis = 1)
merge_df = pd.concat([faa_ntsb_splits[0], moving_avg], axis = 1)

In [ ]:
merge_df.head(10)

In [ ]:
merge_df

### Entire pipeline

In [ ]:
window_len = 6
lag = 1

asrs_splits = [asrs_merge[asrs_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
faa_ntsb_splits = [faa_ntsb_merge[faa_ntsb_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
final = []

for i in range(len(asrs_splits)):
    asrs = asrs_splits[i].reset_index().drop('index', axis = 1)
    faa_ntsb = faa_ntsb_splits[i].reset_index().drop('index', axis = 1)
    
    # compute moving average of ASRS 
    mov_avg = asrs.rolling(window_len, min_periods=window_len).mean()
    
    # insert empty rows at the beginning for lagging
    empty_row_df = pd.DataFrame(data = [pd.Series(index=asrs.columns) for _ in range(lag)])
    empty_row_df = empty_row_df.drop('tracon', axis = 1)   
    
    # merge the moving averages of ASRS with FAA+NTSB 
    moving_avg = pd.concat([empty_row_df, mov_avg])
    moving_avg = moving_avg.drop(['year', 'month'], axis = 1)
    moving_avg = moving_avg.reset_index().drop('index', axis = 1)
    merge_df = pd.concat([faa_ntsb, moving_avg], axis = 1)
    final.append(merge_df)

final_df = pd.concat(final).reset_index().drop('index', axis = 1)
final_df.to_csv('quality_check/final.csv')

In [ ]:
final_df.head(10)

# Quality check

In [ ]:
try:
    os.mkdir("quality_check")
except OSError as error:
    print(error)

np.random.seed(8)   
final_merge.sample(10).to_csv("quality_check/final_merge_10.csv")